In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from pandas.io import gbq

from zipfile import ZipFile

In [3]:
def tbl_exists(client, table_ref):
    from google.cloud.exceptions import NotFound
    try:
        client.get_table(table_ref)
        return True
    except NotFound:
        return False

In [4]:
# These first two values will be different on your machine. 
service_path = '/Users/steviodong/Desktop/Applied Data Analytics/TWP/'
service_file = 'wedge-328901-b7c6a35e5da2.json' # change this to your authentication information  
gbq_proj_id = 'wedge-328901' # change this to your project. 
gbq_dataset_id = 'wedgeclean' # and change this to your data set ID

private_key =service_path + service_file

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

client = bigquery.Client(credentials = credentials, project=gbq_proj_id)



In [5]:
# create a table that returns all columns of distinct card numbers in gbq

my_table = "distinct_cards"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

In [6]:
if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name) 
    
    query_text ="".join(['DELETE FROM `',table_full_name,'` WHERE 1=1'])

    job_config = bigquery.QueryJobConfig()

    query_job = client.query(
        query_text,
        location="US",
        job_config=job_config,
    )  # API request - starts the query

    query_job.result()  # Waits for the query to finish

In [7]:
table = client.get_table(table_ref)
print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))

Table distinct_cards contains 1 columns


In [8]:
job_config = bigquery.QueryJobConfig(destination=table_ref)

# grab all of the members' distinct card numbers
sql =  """
    SELECT distinct(card_no)
    FROM `wedge-328901.wedgeclean.clean_transArchive_*` 
    WHERE card_no != 3
    """
# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))

Query results loaded to the table distinct_cards


In [9]:
# execute a new query to get a random 1.25% sample of owners

my_table = "distinct_cards_sample"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name) 

    query_text ="".join(['DELETE FROM `',table_full_name,'` WHERE 1=1'])

    job_config = bigquery.QueryJobConfig()

    query_job = client.query(
        query_text,
        location="US",
        job_config=job_config,
    )  # API request - starts the query

    query_job.result()  # Waits for the query to finish

job_config = bigquery.QueryJobConfig(destination=table_ref)

sql = """
  SELECT
  *
FROM
  `wedge-328901.wedgeclean.distinct_cards` 
  where rand() < 0.0125
"""

query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))

Query results loaded to the table distinct_cards_sample


In [10]:
# Match all transaction data to the owners in the random sample table

my_table = "owners_sample"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name) 

    query_text ="".join(['DELETE FROM `',table_full_name,'` WHERE 1=1'])
    
    job_config = bigquery.QueryJobConfig()

    query_job = client.query(
        query_text,
        location="US",
        job_config=job_config,
    )  # API request - starts the query

    query_job.result()  # Waits for the query to finish

job_config = bigquery.QueryJobConfig(destination=table_ref)

sql = """
SELECT
  *
FROM
  `wedge-328901.wedgeclean.clean_transArchive_*`
WHERE
  card_no IN (
  SELECT
    *
  FROM
    `wedge-328901.wedgeclean.distinct_cards_sample`)
"""

query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))

Query results loaded to the table owners_sample


In [14]:
# put the results in a dataframe
df = client.query(sql).to_dataframe()
df.head(10)

,datetime,register_no,emp_no,trans_no,upc,description,trans_type,trans_subtype,trans_status,department,...,batchHeaderID,local,organic,display,receipt,card_no,store,branch,match_id,trans_id
0,2014-10-06 17:33:16+00:00,4.0,83.0,31.0,TAX,Tax,A,None,None,0.0,...,None,0.0,NaN,None,0.0,48443.0,1.0,0.0,0.0,39.0
1,2014-10-12 19:39:47+00:00,5.0,4.0,34.0,TAX,Tax,A,None,None,0.0,...,None,0.0,NaN,None,0.0,48340.0,1.0,0.0,0.0,58.0
2,2014-10-19 16:20:37+00:00,19.0,39.0,2.0,TAX,Tax,A,None,None,0.0,...,None,0.0,NaN,None,0.0,48441.0,1.0,0.0,0.0,32.0
3,2014-10-29 15:31:45+00:00,5.0,29.0,71.0,TAX,Tax,A,None,None,0.0,...,None,0.0,NaN,None,0.0,48443.0,1.0,0.0,0.0,37.0
4,2014-11-01 10:53:47+00:00,5.0,84.0,37.0,TAX,Tax,A,None,None,0.0,...,None,0.0,NaN,None,0.0,48346.0,1.0,0.0,0.0,41.0
5,2014-11-03 16:46:21+00:00,6.0,61.0,78.0,TAX,Tax,A,None,None,0.0,...,None,0.0,NaN,None,0.0,48443.0,1.0,0.0,0.0,18.0
6,2014-11-17 14:53:23+00:00,6.0,33.0,8.0,TAX,Tax,A,None,None,0.0,...,None,0.0,NaN,None,0.0,48443.0,1.0,0.0,0.0,21.0
7,2014-10-06 17:33:16+00:00,4.0,83.0,31.0,DISCOUNT,Discount,I,None,None,0.0,...,None,0.0,NaN,None,0.0,48443.0,1.0,0.0,0.0,38.0
8,2014-10-12 19:39:47+00:00,5.0,4.0,34.0,DISCOUNT,Discount,I,None,None,0.0,...,None,0.0,NaN,None,0.0,48340.0,1.0,0.0,0.0,57.0
9,2014-10-19 16:20:37+00:00,19.0,39.0,2.0,DISCOUNT,Discount,I,None,None,0.0,...,None,0.0,NaN,None,0.0,48441.0,1.0,0.0,0.0,31.0


In [12]:
# export as a txt file
df.to_csv('owners_sample.txt', sep= '\t')